Another TPS, weird set, but better than TPS-August. My method is simple and straight forward, something that will help any beginner. 

P.S - Would appreciate constructive criticism

# Importing Data and Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
%matplotlib inline
import lightgbm as lgb
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

font1 = {'family':'serif','color':'brown','size':15}

../input/tabular-playground-series-jan-2021/train.csv

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
sample = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

# ID is of no use to me 
train.drop(columns = 'id', inplace = True)
test.drop(columns = 'id', inplace = True)

In [ ]:
print('Train Shape', train.shape, '\n', 'Test Shape', test.shape)
train.describe().T

# Data Exploration

In [ ]:
# Simple exploration
print('Null in Train', train.isnull().sum().sum())
print('Null in Test', test.isnull().sum().sum())
print('Train Duplicate data', train.duplicated().sum())
print('Test Duplicate data', test.duplicated().sum())

## Univarite analysis
 Box plot and Histogram

In [ ]:
hist = train.hist(figsize = (25, 12), bins=50, grid = False, 
                  xlabelsize=8, ylabelsize=8, layout = (3,5))

In [ ]:
box = train.boxplot(figsize = (18,8), rot = 20 )

Well I will plot this again without Target, as the scales are clearly different

In [ ]:
box = train.drop(columns = ['target']).boxplot(figsize = (18,8), rot = 20 )

**Inference** : 

Not Many outliers. The Scale of the variables is pretty same, still will check if the model accuracy increases with scaling.

In [ ]:
train.columns

In [ ]:
cols = ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14', 'target']
sns.pairplot(train[cols], size = 4)

In [ ]:
plt.figure(figsize=(18,12))
plt.title("Correlation Plot", fontdict = font1)
sns.heatmap(train.corr(), annot=True, cmap="RdBu", fmt='.2f', 
            center = 0, linewidths=0.1, cbar_kws={"shrink": .8}) 

**Inference** :

Not a bad correlation but the data points are all over the place. I'll perform PCA to reduce the feature number. Lets see how PCA will work out with my model(s).

# PCA

PCA is scale sensitive and thus need scaling before

In [ ]:
# Splitting
x = train.iloc[:,:-1]
y = train.iloc[:,-1]


# PCA is scale sensitive and thus need scaling before
# Standard scaling
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
x_train = pd.DataFrame(ss.fit_transform(x))

In [ ]:
# PCA
from sklearn.decomposition import PCA
pca = PCA()
pca_train = pca.fit_transform(x_train)

# Converting PC Datasets for model training
pca_train = pd.DataFrame(pca_train)

list_name = [f'pc{i}' for i in range(14)]

pca_train.columns = list_name

In [ ]:
# NO of PC's
explained_variance = pca.explained_variance_ratio_

plt.plot(explained_variance)
plt.grid()
plt.xlabel('Number of components')
plt.ylabel('Explained variance')
plt.title("Scree Plot");

Well the PCA looks promising and PC's 6-7 looks good, that can explain >80% of variance

In [ ]:
hist = pca_train.hist(figsize = (25, 12), bins=50, 
                      grid = False, xlabelsize=8, ylabelsize=8, 
                      layout = (3,5))
plt.suptitle('PCA-train set', fontsize = 20)

Not much of change in the data distribution, now becoming bit skeptic of PCA in model building.

What I can do is to run a baseline model on PCA and Non PCA data and check accuracy with CV

In [ ]:
# x = pca_train
# y 

# Baseline models - PCA vs Normal Dataset

Here I am evaluating the modle type and the dataset to work on for the final model building. 

Since its a regression problem with not that good correlation a random forest vs LGBM makes sense for now. 

P.S - Any suggestions what other models can be applied, comment down please


## Random Forest

In [ ]:
# prepare the cross-validation procedure
cv = KFold(n_splits = 3, random_state = 2, shuffle=True)

# Import the models we are using
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(verbose = 1)

In [ ]:
# evaluate model for Random Forest with original Dataset
scores = cross_val_score(model, x, y, cv = cv, scoring = "neg_mean_absolute_error")
print('Random Forest baseline score without PCA' , -scores.mean())

In [ ]:
# evaluate model For Random Forest with PCA Dataset
scores = cross_val_score(model, pca_train, y, cv = cv, scoring = "neg_mean_absolute_error")
print('Random Forest baseline score with PCA' , -scores.mean())

## LGBM

In [ ]:
x = train.iloc[:,:-1]
y = train.iloc[:,-1]

In [ ]:
# prepare the cross-validation procedure
cv = KFold(n_splits=3, random_state=2, shuffle=True)

# Import the models we are using
from sklearn.ensemble import RandomForestRegressor
model = lgb.LGBMRegressor()

In [ ]:
# evaluate model
scores = cross_val_score(model, x, y, cv = cv, scoring = "neg_mean_absolute_error")
print('LGBM baseline score without PCA' , -scores.mean())

In [ ]:
# evaluate model
scores = cross_val_score(model, pca_train, y, cv = cv, scoring = "neg_mean_absolute_error")
print('LGBM baseline score with PCA' , -scores.mean())

### **Base Summary**

* Error - Random forest with PCA -> 0.6005048129803532
* Error - Random forest without PCA -> 0.5926412679654879

* Error - LGBM with PCA -> 0.6022344194486661
* Error - LGBM with PCA -> 0.5900531147333651

Well few things that are clear that, modelling with PCA was futile, since the error in the base model was higher in case of PCA.

Also I am finalising LGBM over Random forest, since the runtime was too much for random forest compared to LGBM. Next step will be hypertuning the LGBM Parameters.


# Final Model

## Hyperparameter Tuning

In [ ]:
# Parameters

n_estimators = [100, 300, 500, 700, 900]
num_leaves = [30, 50, 80]
random_state = [2, 5]
# max_depth = [-1, 30, 50, 100]
# learning_rate = [0.1, 0.05, 0.01]

*I have commented out some parameters, since the run time was getting too high exceeding the RAM capacity. 
If anyone has a suggestion for a better way to compute multiple parameters without running out of RAM, please do comment out.*

In [ ]:
# Dictionary of parameters
params = {'n_estimators': n_estimators,
          'num_leaves': num_leaves,
          'random_state': random_state
          } 

In [ ]:
# create model
model = lgb.LGBMRegressor()

r_search = RandomizedSearchCV(estimator = model, param_distributions = params, 
                              cv = 3, scoring ='neg_mean_absolute_error', 
                              n_iter = 10, verbose=1, return_train_score=True)

r_search.fit(x,y)

In [ ]:
print('Best parameters', r_search.best_params_)
print('Best score', -r_search.best_score_)

In [ ]:
rcv_results = pd.DataFrame(r_search.cv_results_)
rcv_results

## Final Showdown

In [ ]:
model = lgb.LGBMRegressor(n_estimator = 900, num_leaves = 30,  )
model.fit(x, y)

In [ ]:
sample['target'] = model.predict(test)
sample.to_csv('submission.csv', index = False)